<a href="https://colab.research.google.com/github/tchih11/qiita/blob/main/notebooks/spacy_scattertext/01_create_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
user_name = 'tchih11'
repository_name = "qiita"

!git clone https://{user_name}@github.com/{user_name}/{repository_name}.git
%cd /content/qiita/notebooks/spacy_scattertext

In [6]:
%%capture
!pip install neologdn

import glob
import os
import pathlib
import tarfile

import neologdn
import pandas as pd

In [7]:
# Livedoorニュースのファイルをダウンロード
! wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

# rawデータ解凍
tar = tarfile.open("./ldcc-20140209.tar.gz", "r:gz")
tar.extractall(".")
tar.close()

--2021-03-22 23:50:30--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz.1’

ldcc-20140209.tar.g 100%[===================>]   8.44M  1.59MB/s    in 9.2s    

2021-03-22 23:50:40 (936 KB/s) - ‘ldcc-20140209.tar.gz.1’ saved [8855190/8855190]



In [8]:
df = pd.DataFrame(columns=["text", "category"])

for file_path in pathlib.Path("./text").glob("**/*.txt"):
    f_path = pathlib.Path(file_path)
    file_name = f_path.name
    category_name = f_path.parent.name

    # 特殊ファイルはスキップ
    if file_name in ["CHANGES.txt", "README.txt", "LICENSE.txt"]:
        continue

    with open(file_path, "r") as f:
        text_all = f.read()
        text_lines = text_all.split("\n")
        url, time, title, *text = text_lines

        # text前処理
        text = "".join(text)
        text = text.strip()
        text = neologdn.normalize(text)
        text = text.replace(' ', '')
        text = text.replace('　', '')

        df.loc[file_name] = [text,category_name]
        
# インデックスに使用していたファイル名を削除
df.reset_index(inplace=True,drop=True)

In [14]:
df.to_pickle("../../data/livedoor_news.pkl")

/content/qiita/notebooks/spacy_scattertext
